In [19]:
import os
import whisper
import polars as pl
import pickle
from babel.dates import format_time

In [20]:
# Function to transcribe audio using OpenAI's transcription service
def transcribe_audio(model, audio_path, transcript_path, df):
    n = df.shape[0]
    temp_column = []
    for i in range(0, n):
        print("Processing segment " + chr(i) + " of " + chr(n))
        file_path = audio_path + "/" + df[i, 'file_id'] + "_" + str(round(df[i, 'start'], 3)) + ".wav"
        result = model.transcribe(file_path)
        temp_column.append(result['text']) # add transcript text to column to add to df
        with open(transcript_path + "/" + df[i, 'file_id'] + "_" + str(round(df[i, 'start'], 3)), "wb") as fp:   # pickling full transcript data
            pickle.dump(result, fp)
        print("Segment " + i + " of " + n + " complete")
    df = df.with_columns(pl.Series(name="transcript", values=temp_column))
    return(df)


In [14]:
df = pl.read_csv("../data/ad_validation_initial.csv")
df_test = df[6:8]
print(df_test)

shape: (2, 7)
┌──────────────────────────────┬─────────────┬────────┬──────────┬───────────┬─────────────┬───────┐
│ file_id                      ┆ speaker_est ┆ start  ┆ duration ┆ end       ┆ speaker_val ┆ notes │
│ ---                          ┆ ---         ┆ ---    ┆ ---      ┆ ---       ┆ ---         ┆ ---   │
│ str                          ┆ str         ┆ f64    ┆ f64      ┆ f64       ┆ str         ┆ str   │
╞══════════════════════════════╪═════════════╪════════╪══════════╪═══════════╪═════════════╪═══════╡
│ pres_trimmed_incl_scene-P-19 ┆ SPEAKER_03  ┆ 33.998 ┆ 8.472    ┆ 42.47     ┆ other       ┆ null  │
│ 04-1…                        ┆             ┆        ┆          ┆           ┆             ┆       │
│ pres_trimmed_incl_scene-P-19 ┆ SPEAKER_02  ┆ 42.47  ┆ 2.088    ┆ 44.558002 ┆ biden       ┆ null  │
│ 04-1…                        ┆             ┆        ┆          ┆           ┆             ┆       │
└──────────────────────────────┴─────────────┴────────┴──────────┴───────────

In [21]:
model = whisper.load_model("large")
transcribe_audio(model, "../data/ad_audio_split_testing", "../data/ad_transcripts_testing", df_test)

TypeError: can only concatenate str (not "int") to str